In [1]:
# Package imports
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score

import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_score

from xgboost import XGBRegressor

In [2]:
# Import data
modeling_data = pd.read_csv("data/final_modeling_data.csv")

# All X vars and the Y variable
only_cols = ['delay_minutes', 'dispatch_hour', 'dispatch_day', 'job_type_encoded', 'traffic_level_encoded', 'temperature_normalized', 'precipitation_normalized']
modeling_data = modeling_data[only_cols]



# statsmodels package basic linear modeling

In [3]:
# Pre-train/test split for Statsmodels package data
X_stats = modeling_data.drop('delay_minutes', axis=1)
y_stats = modeling_data['delay_minutes']

# statsmodels constant term adjustment for intercept variable
X_stats = sm.add_constant(X_stats) 

# Train/test Split
X_train, X_test, y_train, y_test = train_test_split(
    X_stats, y_stats, test_size=0.2, random_state=42
)

# Fit regular regression using statsmodels
lin_mod_sm = sm.OLS(y_train, X_train).fit()
print(lin_mod_sm.summary())

# Create y-hat
# We must add_constant to X_test if not already done
y_pred_test_stats = lin_mod_sm.predict(X_test)

# MSE, MAE, R^2 values for this model
mse_sm = mean_squared_error(y_test, y_pred_test_stats)
mae_sm = mean_absolute_error(y_test, y_pred_test_stats)
r2_sm = r2_score(y_test, y_pred_test_stats)

print("\n=== Test data results ===")
print(f"MAE:  {mae_sm:.2f}")
print(f"MSE:  {mse_sm:.2f}")
print(f"RMSE: {mse_sm**0.5:.2f}")
print(f"R^2:  {r2_sm:.2f}")
# None of thse are significant here which is no surprise since the data is entirely random noise

                            OLS Regression Results                            
Dep. Variable:          delay_minutes   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                    0.6040
Date:                Sun, 26 Jan 2025   Prob (F-statistic):              0.727
Time:                        18:01:27   Log-Likelihood:                -1669.3
No. Observations:                 399   AIC:                             3353.
Df Residuals:                     392   BIC:                             3380.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                   

# Linear regression using sklearn

In [4]:
# X and Y datasets
X = modeling_data.drop('delay_minutes', axis=1)
y = modeling_data['delay_minutes']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Fit model
lin_model = LinearRegression()
lin_model.fit(X_train, y_train)

# predict on test set to find out of sample fit
y_hat_test = lin_model.predict(X_test)

# MSE MAE, R^2 on test data
mse_lin = mean_squared_error(y_test, y_hat_test)
mae_lin = mean_absolute_error(y_test, y_hat_test)
r2_lin = r2_score(y_test, y_hat_test)

print("=== Basic LM using SKLearn ===")
print(f"Test MAE: {mae_lin:.2f}")
print(f"Test MSE: {mse_lin:.2f}")
print(f"Test RMSE: {mse_lin**0.5:.2f}")
print(f"Test R^2:  {r2_lin:.2f}")

# Cross-Validation RMSE
scores_lin = cross_val_score(lin_model, X, y, cv=5, scoring='neg_mean_squared_error')
rmse_cv_lin = (-scores_lin.mean())**0.5
print(f"CV RMSE (5-fold): {rmse_cv_lin:.2f}")


=== Basic LM using SKLearn ===
Test MAE: 13.80
Test MSE: 252.23
Test RMSE: 15.88
Test R^2:  -0.01
CV RMSE (5-fold): 16.12


# Random Forest modeling using sklearn

In [5]:
# RF modeling
# X and Y data
X = modeling_data.drop('delay_minutes', axis=1)
y = modeling_data['delay_minutes']

# X train/test and y train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Fit a RF
model = RandomForestRegressor(n_estimators=100, random_state=642)
model.fit(X_train, y_train)

# Y-hat, MAE, R^2 for our RF model
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"MAE: {mae}, R^2: {r2}")

# CV results here
scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
print(f"Average RMSE: {(-scores.mean())**0.5}")

# Feature importance
importances = model.feature_importances_

print(importances)

MAE: 15.536117619047621, R^2: -0.3381870061630441
Average RMSE: 18.286832577759885
[0.3532407  0.05400296 0.16209874 0.06351174 0.18276129 0.18438456]


# XGB example code in SKLearn

In [6]:
# Create XGB
xgb_model = XGBRegressor(random_state=42)
# Fit model
xgb_model.fit(X_train.drop('const', axis=1, errors='ignore'), y_train)  # dropping 'const' if present
# y hat from xgboost
y_hat_xgb = xgb_model.predict(X_test.drop('const', axis=1, errors='ignore'))

# MAE and R^2 for XGB model
mae_xgb = mean_absolute_error(y_test, y_hat_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)
print(f"XGBoost MAE: {mae_xgb:.2f}, R^2: {r2_xgb:.2f}")

NameError: name 'y_pred_xgb' is not defined